<a href="https://colab.research.google.com/github/anshmehta337/-mnist-digit-classifier/blob/main/fashionmnistcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#here we will try to make our own architecture on MNIST dataset

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transform

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_transform = transform.Compose([
    transform.RandomAffine(
        degrees=10,
        translate=(0.1, 0.1),
        scale=(0.9, 1.1)
    ),
    transform.ToTensor()
])

test_transform = transform.Compose([
    transform.ToTensor()
])

In [ ]:
trainset=torchvision.datasets.FashionMNIST(
    root='./data',
    train=True,
    download=True,
    transform=train_transform
)
testset=torchvision.datasets.FashionMNIST(
    root='./data',
    train=False,
    download=True,
    transform=test_transform
)

In [ ]:
len(trainset)

60000

In [ ]:
train_load=torch.utils.data.DataLoader(trainset,batch_size=64,shuffle=True)
test_loader=torch.utils.data.DataLoader(testset,batch_size=64,shuffle=False)

In [ ]:
import torch.nn as nn

class FashionMNIST_CNN(nn.Module):
    def __init__(self):
        super(FashionMNIST_CNN, self).__init__()

        self.block1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout2d(0.1),
            nn.MaxPool2d(2, 2)  # 64 x 14 x 14
        )

        self.block2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout2d(0.2),
            nn.MaxPool2d(2, 2)  # 128 x 7 x 7
        )

        self.block3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),

            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Dropout2d(0.1),
            nn.MaxPool2d(2, 2)  # 256 x 3 x 3
        )

        self.block4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),

            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Dropout2d(0.3),
            nn.AdaptiveMaxPool2d((1, 1))  # Output: 512 x 1 x 1
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),               # 512 * 1 * 1 = 512
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.classifier(x)
        return x


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = FashionMNIST_CNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
def train(model, train_loader, optimizer, criterion, epoch):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()
        total += targets.size(0)

    print(f'Epoch [{epoch}] - Loss: {total_loss/len(train_loader):.4f} - Accuracy: {100.*correct/total:.2f}%')


In [ ]:
def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            correct += predicted.eq(targets).sum().item()
            total += targets.size(0)

    print(f'Test Accuracy: {100.*correct/total:.2f}%')


In [ ]:
epochs = 25  # You can increase
1
for epoch in range(1, epochs + 1):
    train(model, train_load, optimizer, criterion, epoch)
    evaluate(model, test_loader)


Epoch [1] - Loss: 0.6899 - Accuracy: 73.80%
Test Accuracy: 83.71%
Epoch [2] - Loss: 0.4549 - Accuracy: 83.57%
Test Accuracy: 86.14%
Epoch [3] - Loss: 0.3917 - Accuracy: 85.97%
Test Accuracy: 88.62%
Epoch [4] - Loss: 0.3566 - Accuracy: 87.40%
Test Accuracy: 90.31%
Epoch [5] - Loss: 0.3374 - Accuracy: 88.17%
Test Accuracy: 88.80%
Epoch [6] - Loss: 0.3154 - Accuracy: 88.97%
Test Accuracy: 90.59%
Epoch [7] - Loss: 0.3041 - Accuracy: 89.28%
Test Accuracy: 91.78%
Epoch [8] - Loss: 0.2924 - Accuracy: 89.83%
Test Accuracy: 91.57%
Epoch [9] - Loss: 0.2807 - Accuracy: 90.16%
Test Accuracy: 91.84%
Epoch [10] - Loss: 0.2699 - Accuracy: 90.68%
Test Accuracy: 92.30%
Epoch [11] - Loss: 0.2612 - Accuracy: 90.87%
Test Accuracy: 91.39%
Epoch [12] - Loss: 0.2573 - Accuracy: 91.05%
Test Accuracy: 92.67%
Epoch [13] - Loss: 0.2500 - Accuracy: 91.29%
Test Accuracy: 91.99%
Epoch [14] - Loss: 0.2416 - Accuracy: 91.50%
Test Accuracy: 92.69%
Epoch [15] - Loss: 0.2353 - Accuracy: 91.67%
Test Accuracy: 93.12%
Epoc